In [1]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from env import get_db_url
import env
import seaborn as sns

# local libraries
import wrangle_zillow

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split

In [2]:
df=wrangle_zillow.wrangle_zillow()

Using cached data


In [3]:
df.shape

(77380, 61)

In [4]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77380 non-null  int64  
 1   parcelid                    77380 non-null  int64  
 2   basementsqft                50 non-null     float64
 3   bathroom                    77380 non-null  float64
 4   bedroom                     77380 non-null  float64
 5   condition                   49671 non-null  float64
 6   calculatedbathnbr           76771 non-null  float64
 7   decktypeid                  614 non-null    float64
 8   finishedfloor1squarefeet    6023 non-null   float64
 9   square_feet                 77184 non-null  float64
 10  finishedsquarefeet12        73748 non-null  float64
 11  finishedsquarefeet13        41 non-null     float64
 12  finishedsquarefeet15        3009 non-null   float64
 13  finishedsquarefeet50        602

,id,parcelid,basementsqft,bathroom,bedroom,condition,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,square_feet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,tax_value,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,"77,380.00","77,380.00",50.00,"77,380.00","77,380.00","49,671.00","76,771.00",614.00,"6,023.00","77,184.00",...,"17,560.00",172.00,"77,268.00","77,379.00","77,380.00","77,378.00","77,375.00","2,886.00","77,136.00","77,380.00"
mean,"1,495,126.49","13,007,149.52",679.72,2.30,3.05,6.53,2.32,66.00,"1,366.51","1,785.23",...,1.43,1.00,"189,315.98","490,137.46","2,016.00","301,096.94","5,995.57",14.09,"60,496,735,236,341.99",0.02
std,"860,905.69","3,481,368.13",689.70,1.00,1.14,1.72,0.98,0.00,671.31,954.05,...,0.54,0.00,"230,088.56","653,447.67",0.00,"492,599.03","7,622.89",2.19,"1,535,251,869,244.56",0.17
min,349.00,"10,711,855.00",38.00,0.00,0.00,1.00,1.00,66.00,44.00,128.00,...,1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",-4.66
25%,"752,050.00","11,538,296.75",273.00,2.00,2.00,6.00,2.00,66.00,955.00,"1,182.00",...,1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.59",14.00,"60,373,109,005,001.75",-0.02
50%,"1,497,870.00","12,531,548.50",515.00,2.00,3.00,6.00,2.00,66.00,"1,257.00","1,543.00",...,1.00,1.00,"136,499.50","358,976.00","2,016.00","203,383.50","4,450.72",15.00,"60,376,032,003,008.00",0.01
75%,"2,240,480.25","14,211,843.75",796.50,3.00,4.00,8.00,3.00,66.00,"1,615.00","2,113.00",...,2.00,1.00,"218,787.50","569,003.00","2,016.00","366,802.25","6,927.80",15.00,"60,590,423,251,008.00",0.04
max,"2,982,274.00","167,689,317.00","3,560.00",18.00,16.00,12.00,18.00,66.00,"6,912.00","35,640.00",...,6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00",5.26


### Remove any properties that are not single unit. Don't want to lose all the `NaNs`

In [5]:
df.unitcnt[(df.unitcnt==1)|(df.unitcnt.isna())].value_counts(dropna=False)

                1.00    47292
NaN                     26818
Name: unitcnt, dtype: int64

In [6]:
# #Filter out anything other than unit count = 1 and nans
# df = df[(df.unitcnt==1)|(df.unitcnt.isna())]

In [7]:
# # Keep properties that should be single units
# properties_to_keep = ['Single Family Residential','Condominum','Mobile Home','Manufactured, Modular, Prefabricated Homes','Residential General','Townhouse',np.nan]
# df = df[df.apply(lambda row: row.propertylandusedesc in properties_to_keep, axis=1)]
# df.propertylandusedesc.value_counts(dropna=False)

In [8]:
# Added to wrangle_zillow.py file
# # Just want single unit properties
    
#     # Filter out anything other than unit count = 1 and nans
#     df = df[(df.unitcnt==1)|(df.unitcnt.isna())]
    
#     # Keep properties that should be single units
#     properties_to_keep = ['Single Family Residential','Condominum','Mobile Home','Manufactured, Modular, Prefabricated Homes','Residential General','Townhouse',np.nan]
#     df = df[df.apply(lambda row: row.propertylandusedesc in properties_to_keep, axis=1)]
    
#     # First pass on removing missing values
#     df_nulls_removed = handle_missing_values(df, prop_required_column=0.3, prop_required_row=0.00002)
    
#     # Fill na values and drop specific columns based on exploring nans
#     df_nulls_removed['garage'] = df_nulls_removed['garage'].fillna(0)
#     df_nulls_removed['garagetotalsqft'] = df_nulls_removed['garagetotalsqft'].fillna(0) # 'No garage'
#     df_nulls_removed['poolsizesum'] = df_nulls_removed['poolsizesum'].fillna(0)# 'No pool'
#     df_nulls_removed['basementsqft'] = df_nulls_removed['basementsqft'].fillna(0) # 'No basement information'
#     df_nulls_removed['threequarterbathnbr'] = df_nulls_removed['threequarterbathnbr'].fillna(0)
#     df_nulls_removed['taxdelinquencyyear'] = df_nulls_removed['taxdelinquencyyear'].fillna(0) # "Assumed Not Delinquent"
#     df_nulls_removed['condition'] = df_nulls_removed['condition'].fillna(-1) # "Not available"
#     df_nulls_removed['yardbuildingsqft17'] = df_nulls_removed['yardbuildingsqft17'].fillna(0) # "No Patio Information"
#     df_nulls_removed['yardbuildingsqft26'] = df_nulls_removed['yardbuildingsqft26'].fillna(0) # "No Yard Building"
#     df_nulls_removed = df_nulls_removed.drop(columns = ['regionidneighborhood','calculatedbathnbr','finishedsquarefeet13','finishedsquarefeet50','finishedsquarefeet6','finishedsquarefeet12','finishedfloor1squarefeet'])
#     # Make a column for the county based on FIPS
#     df_nulls_removed["county"] = np.select([df_nulls_removed.fips == 6037, df_nulls_removed.fips==6059, df_nulls_removed.fips == 6111],["Los Angeles County", "Orange County", "Ventura County"])
    
#     # Fill in binary values with 0s
#     for col in df_nulls_removed.columns:
#         if df_nulls_removed[col].nunique() == 1:
#             df_nulls_removed[col] = df_nulls_removed[col].fillna('None')
    
#     # Fill in count, number, and desc values with 0s and not specified
#     for col in df_nulls_removed.columns:
#         if 'desc' in col:
#             df_nulls_removed[col] = df_nulls_removed[col].fillna('Not Specified')
#         elif 'cnt' in col:
#             df_nulls_removed[col] = df_nulls_removed[col].fillna(0)
#         elif 'number' in col:
#             df_nulls_removed[col] = df_nulls_removed[col].fillna(0)
            
#     # For now, just remove remaining null values
#     df_nulls_removed = df_nulls_removed.dropna()
            
#     return df_nulls_removed


In [9]:
# df = wrangle_zillow.handle_missing_zillow_values(df)

In [10]:
wrangle_zillow.nulls_by_column(df)

,n_rows_missing,pct_rows_missing
buildingclassdesc,77365,1.00
finishedsquarefeet13,77339,1.00
basementsqft,77330,1.00
storydesc,77330,1.00
yardbuildingsqft26,77310,1.00
...,...,...
propertycountylandusecode,0,0.00
parcelid,0,0.00
roomcnt,0,0.00
rawcensustractandblock,0,0.00


### Null by column:
- Drop columns with over 90% missing
 - Some categories should be filled with 0 (garage, pool, etc.)
- Need to consider what to do with columns that are categorically represented once (Condition only represented 

In [11]:
wrangle_zillow.nulls_by_row(df)

,num_cols_missing,pct_cols_missing,num_rows
0,28,0.46,20660
1,29,0.48,16097
2,27,0.44,12410
3,26,0.43,7996
4,30,0.49,6391
5,25,0.41,5768
6,31,0.51,3604
7,32,0.52,2089
8,24,0.39,660
9,23,0.38,599


###Takeaway
 - fix issues by columns, then rows may fix values
 - .60 of missing nulls could be eliminated without affecting dataset.


In [12]:
df_nulls_removed = wrangle_zillow.handle_missing_values(df, prop_required_column=0.3, prop_required_row=0.00002)

For threshold based dropping: 
(77380, 61)  original shape
(77380, 61)  shape after dropping columns with prop required rows below theshold
(77380, 61)  shape after dropping rows with prop required columns below threshold


In [13]:
wrangle_zillow.nulls_by_column(df_nulls_removed)

,n_rows_missing,pct_rows_missing
buildingclassdesc,77365,1.00
finishedsquarefeet13,77339,1.00
basementsqft,77330,1.00
storydesc,77330,1.00
yardbuildingsqft26,77310,1.00
...,...,...
propertycountylandusecode,0,0.00
parcelid,0,0.00
roomcnt,0,0.00
rawcensustractandblock,0,0.00


In [14]:
wrangle_zillow.nulls_by_row(df_nulls_removed)

,num_cols_missing,pct_cols_missing,num_rows
0,28,0.46,20660
1,29,0.48,16097
2,27,0.44,12410
3,26,0.43,7996
4,30,0.49,6391
5,25,0.41,5768
6,31,0.51,3604
7,32,0.52,2089
8,24,0.39,660
9,23,0.38,599


#### Need to fill binary columns (`hashottuborspa`, `fireplaceflag`) 

In [15]:
for col in df_nulls_removed.columns:
    if df_nulls_removed[col].nunique() == 1:
        print(col)
        print(df_nulls_removed[col].value_counts(dropna=False))
        df_nulls_removed[col] = df_nulls_removed[col].fillna('None')

decktypeid
NaN                     76766
               66.00      614
Name: decktypeid, dtype: int64
hashottuborspa
NaN                     75842
                1.00     1538
Name: hashottuborspa, dtype: int64
pools
NaN                     61243
                1.00    16137
Name: pools, dtype: int64
pooltypeid10
NaN                     76916
                1.00      464
Name: pooltypeid10, dtype: int64
pooltypeid2
NaN                     76306
                1.00     1074
Name: pooltypeid2, dtype: int64
pooltypeid7
NaN                     62338
                1.00    15042
Name: pooltypeid7, dtype: int64
fireplaceflag
NaN                     77208
                1.00      172
Name: fireplaceflag, dtype: int64
assessmentyear
2,016.00    77380
Name: assessmentyear, dtype: int64
taxdelinquencyflag
NaN    74494
Y       2886
Name: taxdelinquencyflag, dtype: int64
storydesc
NaN         77330
Basement       50
Name: storydesc, dtype: int64


In [16]:
wrangle_zillow.nulls_by_column(df_nulls_removed)

,n_rows_missing,pct_rows_missing
buildingclassdesc,77365,1.00
finishedsquarefeet13,77339,1.00
basementsqft,77330,1.00
yardbuildingsqft26,77310,1.00
architecturalstyledesc,77174,1.00
...,...,...
rawcensustractandblock,0,0.00
parcelid,0,0.00
propertycountylandusecode,0,0.00
pooltypeid7,0,0.00


### Categorical Wrangling
- 'desc' in name should have NaN
- cnt or number in name villed with 0

In [17]:
for col in df_nulls_removed.columns:
    if 'desc' in col:
        # print(df[col].value_counts(dropna=False))
        df_nulls_removed[col] = df_nulls_removed[col].fillna('Not Specified')
    elif 'cnt' in col:
        df_nulls_removed[col] = df_nulls_removed[col].fillna(0)
    elif 'number' in col:
        df_nulls_removed[col] = df_nulls_removed[col].fillna(0)

In [18]:
for col in df_nulls_removed.columns:
    if (df_nulls_removed[col].nunique() < 15) & (df_nulls_removed[col].nunique() > 2) & (df_nulls_removed[col].isna().sum()>0):
        print(col)
        print(df_nulls_removed[col].value_counts(dropna=False))
        # df[col] = df[col].fillna('None')

condition
NaN                     27709
                8.00    14645
                6.00    13462
                4.00     9482
                7.00     5971
                9.00     2299
                5.00     2088
               11.00      931
                3.00      329
               10.00      313
               12.00      116
                1.00       26
                2.00        9
Name: condition, dtype: int64
finishedsquarefeet13
NaN                     77339
            1,440.00       18
            1,344.00        5
            1,536.00        3
            1,488.00        3
            1,152.00        2
            1,392.00        2
            1,248.00        2
            1,056.00        1
            1,300.00        1
            1,560.00        1
            1,512.00        1
            1,060.00        1
            1,200.00        1
Name: finishedsquarefeet13, dtype: int64
garage
NaN                     51906
                2.00    18608
                1.00 

# Other tasks
- Change nas with 0
- nulls changed to "no ----"
- drop duplicate columns

In [19]:
df_nulls_removed['garage'] = df_nulls_removed['garage'].fillna(0)
df_nulls_removed['garagetotalsqft'] = df_nulls_removed['garagetotalsqft'].fillna('No garage')
df_nulls_removed['poolsizesum'] = df_nulls_removed['poolsizesum'].fillna('No pool')
df_nulls_removed['basementsqft'] = df_nulls_removed['basementsqft'].fillna('No basement information')
df_nulls_removed['threequarterbathnbr'] = df_nulls_removed['threequarterbathnbr'].fillna(0)
df_nulls_removed['taxdelinquencyyear'] = df_nulls_removed['taxdelinquencyyear'].fillna("Assumed Not Delinquent")
df_nulls_removed['condition'] = df_nulls_removed['condition'].fillna("Not available")
df_nulls_removed['yardbuildingsqft17'] = df_nulls_removed['yardbuildingsqft17'].fillna("No Patio Information")
df_nulls_removed['yardbuildingsqft26'] = df_nulls_removed['yardbuildingsqft26'].fillna("No Yard Building")
df_nulls_removed = df_nulls_removed.drop(columns = ['calculatedbathnbr','finishedsquarefeet13','finishedsquarefeet50','finishedsquarefeet6','finishedsquarefeet12','finishedfloor1squarefeet'])

In [20]:
for col in df_nulls_removed.columns:
    if df_nulls_removed[col].isna().sum()>0:
        print(col, df_nulls_removed[col].isna().sum())
        df_nulls_removed[col].value_counts(dropna=False)

square_feet 196
finishedsquarefeet15 74371
lot_size 8239
regionidcity 1471
regionidneighborhood 46490
zip 50
yearbuilt 263
tax_value 1
taxamount 5
censustractandblock 244


In [21]:
df_nulls_removed.shape

(77380, 55)

In [22]:
df_nulls_removed.drop(columns = ['regionidneighborhood']).dropna().shape

(2946, 54)

In [23]:
df_nulls_removed[df_nulls_removed.regionidneighborhood.isna()].regionidcity.value_counts(dropna=False)

            5,534.00    1912
           40,227.00    1590
NaN                     1471
           25,218.00    1353
           27,110.00    1107
                        ... 
          118,880.00       5
           13,311.00       5
           10,815.00       4
           36,078.00       1
           21,395.00       1
Name: regionidcity, Length: 169, dtype: int64

In [24]:
df.shape

(77380, 61)

In [25]:
df = wrangle_zillow.handle_missing_zillow_values(df)

For threshold based dropping: 
(52463, 61)  original shape
(52463, 59)  shape after dropping columns with prop required rows below theshold
(52463, 59)  shape after dropping rows with prop required columns below threshold
